In [1]:
import sys,re,csv
sys.path.append('../')
from collections import defaultdict
outputfolder = "/Users/amycweng/DH/SERMONS_APP/db/data"

In [8]:
import re
bible = {}
with open(f'../assets/bible/bible_parts.txt','r') as file:
    lines = file.readlines()
    for idx, line in enumerate(lines):
      if len(re.findall("-",line)) >= 2 and not re.search(" ", line.strip("\n")):
        verse = line.strip("\n")
        text = lines[idx+1].strip("\n").strip(" ")
        if re.search(r"^or else$|^yea$|^moreover$|^[lL]o$|^if otherwise$|^wait$|^and yet true$",text): continue
        if verse not in bible: bible[verse] = []
        bible[verse].append(text)

In [15]:
unique = []
for p_list in bible.values(): 
    unique.extend(p_list)
unique = {phrase:idx for idx, phrase in enumerate(set(all_phrases))}

In [16]:
unique_bible = {idx:[] for idx in unique.values()}
for label, v_list in bible.items():
    for verse in v_list:
        vidx = unique[verse]
        unique_bible[vidx].append(label)

In [17]:
phrases_csv = []
for phrase, vidx in unique.items(): 
    phrases_csv.append({"vidx":vidx, "phrase":phrase})

In [19]:
bible_vindices = []
for label, vlist in bible.items(): 
    seen = []
    label = re.sub("-"," ",label).split(" ")
    label = " ".join(label[:-2]) + " " + label[-2] + ":" + label[-1] + " (KJV)"
    for v in vlist: 
        vidx = unique[v]
        if vidx in seen: continue
        bible_vindices.append({'vidx':vidx, 'verse_id':label})
        seen.append(vidx)

In [20]:
with open(f"{outputfolder}/bible_phrase_indices.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=bible_vindices[0].keys())
    writer.writerows(bible_vindices)

In [21]:
with open(f"{outputfolder}/bible_phrases.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=phrases_csv[0].keys())
    writer.writerows(phrases_csv)

In [22]:
verse_hits = {}
with open('../assets/quotes_paraphrases/pre-Elizabethan.txt','r') as file:
    lines = file.readlines()
    for idx, line in enumerate(lines): 
        if len(re.findall("-",line)) >= 2: 
            phrase = lines[idx+1].strip("\n")
            vidx = unique[phrase]
        elif re.search("\t",line): 
            score, meta, doc = line.strip("\n").split("\t")[1:]
            score = float(score)
            meta = meta.strip("(|)").split(", ")
            tcpID = meta[-1].strip("'")
            sidx = meta[0]
            if meta[1] == '-1': loc = "In-Text" 
            else: loc = f"Note {meta[1]}"
            key = (tcpID,sidx,loc,vidx)
            if key in verse_hits: 
                freq = verse_hits[key][1]+1
                verse_hits[key] = (score, freq)
            else: 
                verse_hits[(tcpID,sidx,loc,vidx)] = (score,1)
hit_csv = []
for hit, info in verse_hits.items(): 
    hit_csv.append({"tcpID":hit[0], "sidx": hit[1], 'loc':hit[2],
                    "vidx":hit[3] , "score":info[0],"freq":info[1]})

In [23]:
with open(f"{outputfolder}/possible_quotes_paraphrases.csv","w+") as outfile: 
    writer = csv.DictWriter(outfile, fieldnames=hit_csv[0].keys())
    writer.writerows(hit_csv)